In [1]:
import os
import math
import random
import pandas as pd
from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.image as img

from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from sklearn.model_selection import train_test_split

In [2]:
# !jupyter nbextension enable --py widgetsnbextension

In [18]:
TRAIN_DIR = '/opt/ml/input/data/train/images'
TEST_DIR = '/opt/ml/input/data/eval/'

In [5]:
data = pd.read_csv('./train_data.csv')
data.head()

,image_path,label
0,/opt/ml/input/data/train/images/006621_male_As...,0
1,/opt/ml/input/data/train/images/006621_male_As...,6
2,/opt/ml/input/data/train/images/006621_male_As...,0
3,/opt/ml/input/data/train/images/006621_male_As...,0
4,/opt/ml/input/data/train/images/006621_male_As...,12


In [6]:
train_data_split, val_data_split = train_test_split(
    data, 
    test_size=0.2, 
    shuffle=True, 
    stratify=data['label'],
    random_state=34,
)

In [7]:
print(train_data_split.shape)
print(val_data_split.shape)
train_data_split.shape, val_data_split.shape

(15120, 2)
(3780, 2)


((15120, 2), (3780, 2))

In [8]:
class CustomDataSet(Dataset):
    def __init__(self, image_data, transform):
        self.img_path = image_data['image_path']
        self.label = image_data['label']
        self.transform = transform
    
    def __getitem__(self, idx):
        image = Image.open(self.img_path.iloc[idx])
        label = self.label.iloc[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)
    
    def __len__(self):
        return len(self.label)

In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((300, 300)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [10]:
BATCH_SIZE = 64
EPOCHS = 20
LEARNING_RATE = 0.0002
NUM_CLASSES = 18

In [11]:
train_data = CustomDataSet(train_data_split, transform=transform)
val_data = CustomDataSet(val_data_split, transform=transform)

train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

In [12]:
next(iter(train_data_loader))[0].shape

torch.Size([64, 3, 300, 300])

In [13]:
resnet18 = torchvision.models.resnet18(pretrained=True)

In [14]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18.parameters(), lr=LEARNING_RATE)

loader_type = {
    "train": train_data_loader,
    "test": val_data_loader
}

In [15]:
resnet18.fc = torch.nn.Linear(in_features = 512, out_features=NUM_CLASSES, bias=True)

torch.nn.init.xavier_uniform_(resnet18.fc.weight)
stdv = 1. / math.sqrt(resnet18.fc.weight.size(1))
resnet18.fc.bias.data.uniform_(-stdv, stdv)

resnet18.fc.weight.shape[0]

18

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [17]:
resnet18.to(device)
best_test_acc = 0
best_test_loss = 9999.
patience = 0

for epoch in range(EPOCHS):
    
    for mode in ["train", "test"]:
        running_loss = 0
        running_acc = 0
        
        if mode == 'train':
            resnet18.train()
        elif mode == 'test':
            resnet18.eval()
        
        for idx, (images, labels) in enumerate(loader_type[mode]):
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(mode == 'train'):
                logits = resnet18(images)
                _, preds = torch.max(logits, 1)
                loss = criterion(logits, labels)
                
                if mode == 'train':
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            running_acc += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(loader_type[mode].dataset)
        epoch_acc = running_acc / len(loader_type[mode].dataset)
        
        print(f'epoch: {epoch+1}, {mode}-데이터 셋 Loss: {epoch_loss:.3f}, acc: {epoch_acc:.3f}')
        if mode == 'test' and best_test_acc < epoch_acc:
              best_test_acc = epoch_acc
        if mode == 'test' and best_test_loss > epoch_loss:
              best_test_loss = epoch_loss
        elif mode == 'test' and best_test_loss < epoch_loss:
            patience += 1
    
    if patience == 4:
        print('finished by early stopping!!')
        break
        
print("training end!!")
print(f"best acc: {best_test_acc}, best loss: {best_test_loss}")
              

epoch: 1, train-데이터 셋 Loss: 0.396, acc: 0.876
epoch: 1, test-데이터 셋 Loss: 0.163, acc: 0.943
epoch: 2, train-데이터 셋 Loss: 0.068, acc: 0.980
epoch: 2, test-데이터 셋 Loss: 0.193, acc: 0.934
epoch: 3, train-데이터 셋 Loss: 0.045, acc: 0.987
epoch: 3, test-데이터 셋 Loss: 0.114, acc: 0.961
epoch: 4, train-데이터 셋 Loss: 0.028, acc: 0.992
epoch: 4, test-데이터 셋 Loss: 0.039, acc: 0.987
epoch: 5, train-데이터 셋 Loss: 0.007, acc: 0.998
epoch: 5, test-데이터 셋 Loss: 0.049, acc: 0.983
epoch: 6, train-데이터 셋 Loss: 0.019, acc: 0.995
epoch: 6, test-데이터 셋 Loss: 0.120, acc: 0.960
epoch: 7, train-데이터 셋 Loss: 0.049, acc: 0.984
epoch: 7, test-데이터 셋 Loss: 0.156, acc: 0.950
finished by early stopping!!
training end!!
best acc: 0.9870370626449585, best loss: 0.03905134182324801


In [25]:
# 모델 w, b만 저장
torch.save(resnet18.state_dict(), f'batch_size_{BATCH_SIZE}_epoch_{EPOCHS}_learning_rate_{LEARNING_RATE}.pt')

In [19]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        
        return image

    def __len__(self):
        return len(self.img_paths)

In [22]:
submission = pd.read_csv(os.path.join(TEST_DIR, 'info.csv'))
image_dir = os.path.join(TEST_DIR, 'images')

image_paths = [os.path.join(image_dir, img_path) for img_path in submission['ImageID']]

test_data = TestDataset(image_paths, transform=transform)
test_data_loader = DataLoader(test_data, shuffle=False)

In [24]:
resnet18.eval()

all_predictions = []

for images in test_data_loader:
    with torch.no_grad():
        images = images.to(device)
        pred = resnet18(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

submission.to_csv(f'{TEST_DIR}/submission.csv', index=False)

print('successfully done!')

successfully done!
